<a href="https://colab.research.google.com/github/Deadkiller43/Langchain/blob/main/Task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install langchain_community

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms import HuggingFacePipeline

class ModelLoader:
    def __init__(self, model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0"):
        self.model_name = model_name

    def load(self):
        print("Loading model...")
        tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        model = AutoModelForCausalLM.from_pretrained(self.model_name)

        hf_pipeline = pipeline("text-generation",model=model,tokenizer=tokenizer)

        llm = HuggingFacePipeline(pipeline=hf_pipeline)
        print("Model loaded successfully.")
        return llm

In [8]:
from collections import deque
class ChatMemory:
    def __init__(self, max_turns=3):
        self.memory = deque(maxlen=max_turns)

    def add(self, user_input, bot_reply):
        self.memory.append((user_input, bot_reply))

    def get_context(self):
        return "\n".join([f"User: {u}\nAssistant: {b}" for u, b in self.memory])


In [9]:
from langchain.prompts import PromptTemplate

class ChatInterface:
    def __init__(self):
        self.llm = ModelLoader().load()
        self.memory = ChatMemory()

    def run(self):
        print("Chatbot started. Type /exit to quit.")
        while True:
            user_input = input("User: ")
            if user_input.strip().lower() == "/exit":
                print("Exiting chatbot. Goodbye!")
                break
            history = self.chat_memory.get_context()
            prompt = f"{history}\nUser: {user_input}\nAssistant:"

            output = self.llm(prompt, max_new_tokens=100, temperature=0.7)
            full_text = output[0]["generated_text"]
            bot_reply = full_text[len(prompt):].strip().split("\n")[0]

            print("Bot:", bot_reply)
            self.chat_memory.add(user_input, bot_reply)


In [ ]:
model_loader = ModelLoader()
llm = model_loader.load()

memory = ChatMemory().get_memory()

chatbot = ChatInterface()
chatbot.run()